### Konstantina Andronikou

## Transitive Vs Intransitive Verbs 

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Intransitive 

In [4]:
# remove warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Carla pays")

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Carla pays"]
pred = predict_and_conf(d)

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays', meta=True, nsamples=1000, remove_duplicates = True)

# for k, v in t.items():
#     print(k,v)

In [12]:
with open('Dataset_BiLSTM/Intransitive.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Intransitive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    7 (0.7%)

Example fails:
[ARG1: Lawrence] [V: pays]
----
[ARG1: Alexandra] [V: pays]
----
[ARG1: Lawrence] [V: pays]
----


In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Intransitive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive 

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [16]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [17]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [18]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [19]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
#     print(k, v)

In [20]:
with open('Dataset_BiLSTM/Transitive.txt', 'w') as f:
    print(t.data, file = f)

In [21]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Transitive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    25 (2.5%)

Example fails:
[ARG1: Sally] [V: pays] [ARG3: for coffee] .
----
[ARG1: Alison] [V: pays] [ARG3: for coffee] .
----
[ARG1: Margaret] [V: pays] [ARG3: for coffee] .
----


In [22]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Transitive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive-Intransitive Robustness Test

In [23]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

In [24]:
with open('Dataset_BiLSTM/Intransitive_Rob.txt', 'w') as f:
    print(t.data, file = f)

In [25]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Intransitive_Rob.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    337 (33.7%)

Example fails:
[ARGM-MNR: Ian] [V: payed] .
----
Grace [V: payed] .
----
[ARG1: Bobby] [V: payed] .
----


In [26]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Intransitive_Rob.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive

In [28]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
    #print(k, v)

In [29]:
with open('Dataset_BiLSTM/Transitive_Rob.txt', 'w') as f:
    print(t.data, file = f)

In [30]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Transitive_Rob.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    494 (49.4%)

Example fails:
[ARGM-MNR: Matt] [V: payed] [ARG1: for coffee] .
----
Jessica [V: payed] [ARG1: for coffee] .
----
[ARGM-MNR: Alex] [V: payed] [ARG1: for coffee] .
----


In [31]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Transitive_Rob.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 